<a href="https://colab.research.google.com/github/Rick279a/Inteligencia_Artificial_Il/blob/main/Q_Learning_com_redes_neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q-Learning é uma técnica de aprendizado de máquina que usa uma rede neural convolucional como aprendizado por reforço para ensinar uma inteligência artificial a ter sucesso em um ambiente, obtendo apenas imagens dele como entrada.

As principais diferenças entre o Q-Learning matematico e o Q-Learning com redes neurais estão relacionadas à representação dos estados, uso da memória, escalabilidade e complexidade de implementação.

In [ ]:
import numpy as np
import gym
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from collections import deque
import random

# Configuração do ambiente
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 64
n_episodes = 500
max_steps = 200

# Parâmetros do DQN
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001

# Memória de replay
memory = deque(maxlen=1000)

# Construindo o modelo da rede neural (com menos neurônios)
def build_model():
    model = Sequential()
    model.add(Dense(16, input_dim=state_size, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    return model

# Função para armazenar as experiências
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

# Escolha de ação usando epsilon-greedy
def act(state, model, epsilon):
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    act_values = model.predict(state)
    return np.argmax(act_values[0])

# Replay da memória para treinar a rede
def replay(model, batch_size):
    global epsilon
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
        target = reward
        if not done:
            target = reward + gamma * np.amax(model.predict(next_state)[0])
        target_f = model.predict(state)
        target_f[0][action] = target
        model.fit(state, target_f, epochs=1, verbose=0)
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# Inicializando o modelo
model = build_model()

# Loop de treinamento
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])

    for time in range(max_steps):
        action = act(state, model, epsilon)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"episódio: {e}/{n_episodes}, score: {time}")
            break
        replay(model, batch_size)